In [1]:
import os
import operator

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

In [2]:
dataset_root = os.path.abspath(os.path.expanduser('../2021-TBD/datasets/fashion_mnist'))
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
train_data = image_generator.flow_from_directory(os.path.join(dataset_root, 'train'),
                                                 target_size=(32, 32), shuffle=True)
test_data = image_generator.flow_from_directory(os.path.join(dataset_root, 'test'),
                                                target_size=(32, 32), shuffle=True)
labels = [item[0] for item in sorted(list(train_data.class_indices.items()), key=operator.itemgetter(1))]
print(f'Labels: {labels}')

Found 60000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Labels: ['Ankle boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'T-shirt', 'Trouser']


## 일반적인 텐서플로우 10진 분류

In [3]:
model = tf.keras.models.Sequential([
                       tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
                       tf.keras.layers.MaxPooling2D((2, 2)),
                       tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                       tf.keras.layers.MaxPooling2D((2, 2)),
                       tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                       tf.keras.layers.Flatten(),
                       tf.keras.layers.Dense(64, activation='relu'),
                       tf.keras.layers.Dense(len(labels))])
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                6

In [4]:
epochs = 10
history = model.fit(train_data,
                    epochs=epochs,
                    validation_data=test_data)

Epoch 1/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.4654 - accuracy: 0.8312 - val_loss: 0.3537 - val_accuracy: 0.8704
Epoch 2/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.3037 - accuracy: 0.8894 - val_loss: 0.2912 - val_accuracy: 0.8935
Epoch 3/10
1875/1875 [==============================] - 27s 15ms/step - loss: 0.2631 - accuracy: 0.9035 - val_loss: 0.2849 - val_accuracy: 0.8966
Epoch 4/10
1875/1875 [==============================] - 27s 14ms/step - loss: 0.2319 - accuracy: 0.9130 - val_loss: 0.3071 - val_accuracy: 0.8867
Epoch 5/10
1875/1875 [==============================] - 27s 15ms/step - loss: 0.2065 - accuracy: 0.9240 - val_loss: 0.2575 - val_accuracy: 0.9053
Epoch 6/10
1875/1875 [==============================] - 27s 15ms/step - loss: 0.1854 - accuracy: 0.9302 - val_loss: 0.2812 - val_accuracy: 0.9031
Epoch 7/10
1875/1875 [==============================] - 27s 15ms/step - loss: 0.1672 - accuracy: 0.9380 - val_loss: 0.2684 -

## FLHub API 활용

In [5]:
import flhub

In [6]:
model, version, labels = flhub.download_model('CNN-FashionMNIST')
print(version, labels)
model.summary()

v0.0.51 ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
___________

In [7]:
epochs = 10
history = model.fit(train_data,
                    epochs=epochs,
                    validation_data=test_data)

Epoch 1/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.3554 - accuracy: 0.8812 - val_loss: 0.3069 - val_accuracy: 0.8874
Epoch 2/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.1926 - accuracy: 0.9286 - val_loss: 0.2825 - val_accuracy: 0.9054
Epoch 3/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.1664 - accuracy: 0.9379 - val_loss: 0.2872 - val_accuracy: 0.9056
Epoch 4/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.1522 - accuracy: 0.9437 - val_loss: 0.2995 - val_accuracy: 0.9038
Epoch 5/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.1360 - accuracy: 0.9496 - val_loss: 0.2727 - val_accuracy: 0.9116
Epoch 6/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.1213 - accuracy: 0.9540 - val_loss: 0.3020 - val_accuracy: 0.9050
Epoch 7/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.1108 - accuracy: 0.9589 - val_loss: 0.3515 -

In [8]:
res = flhub.upload_model(model, 'CNN-FashionMNIST', version)
res

'upload completed CNN-FashionMNIST v0.0.51'

## TensorFlow Transfer Learning

In [ ]:
dataset_root = os.path.abspath(os.path.expanduser('../2021-TBD/datasets/fashion_mnist'))
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
train_data = image_generator.flow_from_directory(os.path.join(dataset_root, 'train'),
                                                 target_size=(224, 224), shuffle=True)
test_data = image_generator.flow_from_directory(os.path.join(dataset_root, 'test'),
                                                target_size=(224, 224), shuffle=True)
labels = [item[0] for item in sorted(list(train_data.class_indices.items()), key=operator.itemgetter(1))]
print(f'Labels: {labels}')

In [ ]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
inception_v3 = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"
feature_extractor_model = mobilenet_v2
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)
model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(len(labels))
])
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

In [ ]:
epochs = 10
history = model.fit(train_data,
                    epochs=epochs,
                    validation_data=test_data)